## Transform string dataset into vectors with SISTER

In [15]:
%pip install --upgrade setuptools
%pip install sister

Note: you may need to restart the kernel to use updated packages.

  Using cached sister-0.2.2-py3-none-any.whl (5.3 kB)
     ------------------------------------ 674.8/674.8 kB 134.6 kB/s eta 0:00:00
  Using cached fasttext-0.9.2.tar.gz (68 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached mecab-python3-0.996.5.tar.gz (65 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  error: subprocess-exited-with-error
  
  python setup.py egg_info did not run successfully.
  exit code: 1
  
  [46 lines of output]
  C:\Users\nikol\AppData\Local\Programs\Python\Python310\lib\site-packages\setuptools\installer.py:27: SetuptoolsDeprecationWarning: setuptools.installer is deprecated. Requirements should be satisfied by a PEP 517 installer.
    warnings.warn(
  running egg_info
  creating C:\Users\nikol\AppData\Local\Temp\pip-pip-egg-info-dz36yn9u\mecab_python3.egg-info
  writing C:\Users\nikol\AppData\Local\Temp\pip-pip-egg-info-dz36yn9u\mecab_python3.egg-info\PKG-INFO
  writing dependency_links to C:\Users\nikol\AppData\Local\Temp\pip-pip-egg-info-dz36yn9u\mecab_python3.egg-info\dependency_links.txt
  writing top-level names to C:\Users\nikol\AppData\Local\Temp\pip-pip-egg-info-dz36yn9u\mecab_python3.egg-info\top_level.txt
  writing manifest file 'C:\Users\nikol\AppData\Local\Temp\pip-pip-egg-info-dz36yn9u\mecab_python3.egg-info\SOURCES.txt'
  Traceback (most recent

In [14]:
import sister

sentence_embedding = sister.MeanEmbedding(lang="en")

ModuleNotFoundError: No module named 'sister'

In [13]:
%python --version

UsageError: Line magic function `%python3` not found (But cell magic `%%python3` exists, did you mean that instead?).


# ABT-BUY as vectors

In [27]:
import numpy as np
import pandas as pd 
d1 = pd.read_csv("./D2Aemb.csv", sep='|')
d2 = pd.read_csv("./D2Bemb.csv", sep='|')
gt = pd.read_csv("./D2groundtruth.csv", sep='|')

In [69]:
d2 = d2.replace('\n','', regex=True)
vectors2 = []
for v in d2['Embedded Name']:
    # print(v)
    v = v.replace('[','').replace(']','')
    vv = v.split()
    npv = np.array(vv).astype(np.float32)
    vectors2.append(npv)
vectors2 = np.array(vectors)

In [70]:
d1 = d1.replace('\n','', regex=True)
vectors = []
for v in d1['Embedded Name']:
    # print(v)
    v = v.replace('[','').replace(']','')
    vv = v.split()
    npv = np.array(vv).astype(np.float64)
    vectors.append(npv)
vectors = np.array(vectors)

In [51]:
vectors.shape

(1076, 300)

# FAISS

In [2]:
%pip install faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [53]:
import faiss
import numpy as np 

In [71]:
dimension = vectors.shape[1]    # dimensions of each vector                         
n = vectors.shape[0]    # number of vectors                   
np.random.seed(1)             
db_vectors = np.random.random((n, dimension)).astype('float32')

In [72]:
nlist = 5  # number of clusters
quantiser = faiss.IndexFlatL2(dimension)
index = faiss.IndexIVFFlat(quantiser, dimension, nlist,   faiss.METRIC_L2)

In [73]:
print(index.is_trained)   # False
index.train(db_vectors)  # train on the database vectors
print(index.ntotal)   # 0
index.add(db_vectors)   # add the vectors and update the index
print(index.is_trained)  # True
print(index.ntotal)   # 200

False
0
True
1076


In [74]:
nprobe = 2  # find 2 most similar clusters
n_query = 10  
k = 10  # return 3 nearest neighbours
np.random.seed(0)   
query_vectors = np.random.random((n_query, dimension)).astype('float32')
distances, indices = index.search(vectors2, k)

In [75]:
query_vectors

array([[0.5488135 , 0.71518934, 0.60276335, ..., 0.8621915 , 0.97291946,
        0.9608347 ],
       [0.9065555 , 0.7740473 , 0.33314514, ..., 0.47064894, 0.37824517,
        0.97952694],
       [0.17465839, 0.327988  , 0.6803487 , ..., 0.6277348 , 0.40142795,
        0.41669175],
       ...,
       [0.23040031, 0.4266914 , 0.6104897 , ..., 0.4358412 , 0.7385922 ,
        0.52658427],
       [0.8866832 , 0.8309088 , 0.03160544, ..., 0.60916406, 0.7759584 ,
        0.07614516],
       [0.15805973, 0.6650816 , 0.7384024 , ..., 0.3236137 , 0.813545  ,
        0.6974004 ]], dtype=float32)

In [76]:
distances

array([[91.65165 , 92.4441  , 93.57521 , ..., 95.98072 , 96.1364  ,
        96.48536 ],
       [94.56267 , 95.088905, 96.47396 , ..., 98.737305, 98.962   ,
        99.2308  ],
       [90.785904, 91.561615, 92.49424 , ..., 95.432144, 95.44253 ,
        95.54106 ],
       ...,
       [89.67145 , 91.95996 , 92.3463  , ..., 94.61924 , 94.689   ,
        95.30856 ],
       [89.366714, 91.7596  , 92.51293 , ..., 94.175644, 94.33939 ,
        94.76152 ],
       [88.92613 , 90.67951 , 91.15164 , ..., 94.01607 , 94.25579 ,
        94.999245]], dtype=float32)

In [77]:
indices

array([[625, 550, 649, ..., 126,  21, 787],
       [625, 550, 649, ...,  21, 126,  14],
       [625, 550, 649, ..., 724,  14,  21],
       ...,
       [625, 224, 550, ..., 787,  14,  21],
       [625, 224, 550, ..., 126, 787, 724],
       [625, 550, 649, ..., 724, 948,  14]], dtype=int64)